<a href="https://colab.research.google.com/github/flaviabiaty/analise_cancelamento/blob/main/CancelamentoClientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Projeto de Análise de Cancelamento de Clientes**

Análise de uma base de dados de clientes para verificar o percentual de cancelamentos.

###Importação e tratamento dos dados

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/flaviabiaty/analise_cancelamento/refs/heads/main/cancelamentos_sample.csv")
df.head()

**INFORMAÇÕES DA BASE DE DADOS - CLIENTES**

 - Idade
 - Sexo – Masculino e Feminino
 - Tempo_como_cliente
 - Frequencia_uso –  A frequência de uso desse cliente
 - Ligacoes_callcenter – Quantas vezes o cliente ligou ao Call Center da empresa
 - Dias_atraso – Dias em que o cliente esteve em atraso
 - Assinatura – O plano que o cliente possui
 - Duracao_contrato – Tempo de duração do contrato (Mensal, Trimestral e  Anual)
 - Total_gasto Meses_ultima_interação – Meses desde a última
   interação do cliente
 - Cancelou – Se o cliente cancelou (1) ou não (0) sua assinatura

In [ ]:
#remover a coluna CustomerID
df = df.drop('CustomerID', axis=1)
display(df)

In [ ]:
df.info()

In [ ]:
#campos vazios:
df.isnull().sum()

In [ ]:
linhas_excluir = df[df.isnull().any(axis=1)]
linhas_excluir

In [ ]:
#remoção das linhas com informações em branco
df = df.dropna()
df.info()

###Análise dos dados

Qual o índice de cancelamento dos clientes?

In [ ]:
print(f'Dos {df.shape[0]} clientes na base, {(df["cancelou"]==1.0).sum()} cancelaram. Isso representa {((df["cancelou"]==1.0).sum()/df.shape[0]*100):.2f}%.')

Qual a duração dos contratos dos clientes?

In [ ]:
df['duracao_contrato'].value_counts(normalize=True).map('{:.1%}'.format)

In [ ]:
#analisando os contratos mensais
df.groupby('duracao_contrato').mean(numeric_only=True)

⚡Observa-se que a média de cancelamento dos planos mensais é 1, portanto todos os clientes com planos mensais cancelaram o serviço

In [ ]:
#desconsiderando os planos mensais:
df = df[df['duracao_contrato']!='Monthly']
df['cancelou'].value_counts(normalize=True).map('{:.1%}'.format)

⚡Se desconsiderarmos os planos mensais, a taxa de cancelamento cai para 46,1%

Qual o tipo de assinatura que os clientes realizam?

In [ ]:
display(df['assinatura'].value_counts(normalize=True).map('{:.1%}'.format))
display(df.groupby('assinatura').mean(numeric_only=True))

Os perfis são muito parecidos nos três tipos de assinatura, então esta variável não é tão relevante pra análise

In [ ]:
import plotly.express as px

In [ ]:
#criação de histogramas para todas as colunas em relação ao cancelamento.
for coluna in df.columns:
  grafico = px.histogram(df, x=coluna, color='cancelou')
  grafico.show()

⚡clientes com mais de 20 dias de atraso cancelaram

⚡clientes com mais de 5 ligações ao call center cancelaram

In [ ]:
#desconsiderando os fatores críticos apontados acima:
df = df[df['ligacoes_callcenter']<5]
df = df[df['dias_atraso']<=20]
df['cancelou'].value_counts(normalize=True).map('{:.1%}'.format)


Taxa inicial de cancelamento: **56,7%**

primeiro problema identificado - *planos mensais.*

Se descartarmos esse tipo de plano, a taxa de cancelamento cairia para: **46,1%**

segundo problema identificado - *ligações no call center por mais de 5 vezes*

terceiro problema identificado - *clientes com mais de 20 dias de atraso*

Descartando esses perfis de clientes, a taxa iria para **18,4%**

Outros fatores identificados na análise dos gráficos:

⚡clientes com mais de 50 anos cancelaram

In [ ]:
#desconsiderando os fatores críticos apontados acima:
df = df[df['idade']<50]
df['cancelou'].value_counts(normalize=True).map('{:.1%}'.format)

⚡clientes que gastam menos de R$500 cancelaram

In [ ]:
#desconsiderando os fatores críticos apontados acima:
df = df[df['total_gasto']>500]
df['cancelou'].value_counts(normalize=True).map('{:.1%}'.format)

Alguns fatores, como valor gasto, podem não fazer sentido de serem excluídos da base.

## Possíveis soluções para redução da taxa de cancelamento

Desconto nos planos trimestral e anual, uma vez que todos os clientes do contrato mensal cancelaram

Criação de um alerta quando o cliente ligar a 3ª vez pro callcenter

Criação de um alerta pro time de cobrança quando o cliente bater 10 dias de atraso.

**Resolvendo esses fatores, pode ser obtida uma meta de 20% de cancelamento, muito melhor que os 50% iniciais**